In [1]:
import io
import csv
import os.path

import numpy as np
import numexpr as ne
import matplotlib.pyplot as plt

%matplotlib qt5

In [2]:
blocksPerFile = 6
samplesPerRamp = 6000
rampsPerFile = 1500
samplesPerFile = 9000000
T_files = samplesPerFile / 10e6

In [48]:
#EKF NAV
idx_timeStamp = 0
idx_velNorth = 1
idx_velEast = 2
idx_velDown = 3
idx_velNorth_StdDev = 4
idx_velEast_StdDev = 5
idx_velDown_StdDev = 6
idx_Lat = 7
idx_Long = 8
idx_Alt = 9
idx_undulation = 10
idx_Lat_StdDev = 11
idx_Long_StdDev = 12
idx_Alt_StdDev = 13
idx_nav_status = 14

Lat_1_0  = 48.06069 
Long_1_0 = -1.99354
Lat_1_1  = 48.05507 
Long_1_1 = -2.02359

Lat_2_0  = 48.06249 
Long_2_0 = -1.99467
Lat_2_1  = 48.05687 
Long_2_1 = -2.02434

track3_Lat_0  = 48.06555 
track3_Long_0 = -1.99619
track3_Lat_1  = 48.06007 
track3_Long_1 = -2.02550

runaway = np.array([ [ 48.057546, -2.010483, 0.0 ],
          [ 48.058403, -2.005964, 0.0 ],
          [ 48.058191, -2.005869, 0.0 ],
          [ 48.057327, -2.010383, 0.0 ],
          [ 48.057546, -2.010483, 0.0 ]
          ])

## Load navigation data SbgLogEkfNav

In [4]:
prefix = "/home/pleroy/DATA/PoSAR-MC_PIMA_TEST-2/2017_09_21-22_SBG/dataLogger"

logEkfNav_1  = prefix + "/session_0001/2017_09_22/09h00/" + "sbgLogEkfNav.dat"
logUtcData_1 = prefix + "/session_0001/2017_09_22/09h00/" + "sbgLogUtcData.dat"

logEkfNav_2  = prefix + "/session_0001/2017_09_22/10h00/" + "sbgLogEkfNav.dat"
logUtcData_2 = prefix + "/session_0001/2017_09_22/10h00/" + "sbgLogUtcData.dat"

Load logEkfNav data 09h00

In [5]:
# logEkfNav 09h00
timeStamps_1 = np.loadtxt( logEkfNav_1, skiprows = 1, usecols = idx_timeStamp )
Long_1 = np.loadtxt( logEkfNav_1, skiprows = 1, usecols = idx_Long )
Lat_1 = np.loadtxt( logEkfNav_1, skiprows = 1, usecols = idx_Lat )
Alt_1 = np.loadtxt( logEkfNav_1, skiprows = 1, usecols = idx_Alt )
vN, vE, vD = np.loadtxt( logEkfNav_1, skiprows = 1, usecols = (idx_velNorth, idx_velEast, idx_velDown), unpack=True )

In [6]:
Vel_1 = (vN**2 + vE**2 + vD**2)**0.5

Load logEkfNav data 10h00

In [7]:
# logEkfNav 10h00
timeStamps_2 = np.loadtxt( logEkfNav_2, skiprows = 1, usecols = idx_timeStamp )
Long_2 = np.loadtxt( logEkfNav_2, skiprows = 1, usecols = idx_Long )
Lat_2 = np.loadtxt( logEkfNav_2, skiprows = 1, usecols = idx_Lat )
Alt_2 = np.loadtxt( logEkfNav_2, skiprows = 1, usecols = idx_Alt )
vN_2, vE_2, vD_2 = \
    np.loadtxt( logEkfNav_2, skiprows = 1, usecols = (idx_velNorth, idx_velEast, idx_velDown), unpack=True )

In [8]:
Vel_2 = (vN_2**2 + vE_2**2 + vD_2**2)**0.5

In [9]:
# concatenate 09h00 and 10h00 data
timeStamps = np.concatenate( (timeStamps_1, timeStamps_2) )
Long = np.concatenate( (Long_1, Long_2) )
Lat = np.concatenate( (Lat_1, Lat_2) )
Alt = np.concatenate( (Alt_1, Alt_2) )
Vel = np.concatenate( (Vel_1, Vel_2) )

### Plot navigation data jointly with track references

In [10]:
def plotRunaway( ax ):
    ax.plot(runaway[:,1], runaway[:,0], "og", markeredgecolor = 'black')

In [11]:
def plotLongLatAndTrackReferences( ax ):
    ax.plot( Long, Lat, 'gray' )
    ax.plot( [Long_1_0, Long_1_1], [Lat_1_0, Lat_1_1], "o-.b", markeredgecolor = 'black' )
    ax.plot( [Long_2_0, Long_2_1], [Lat_2_0, Lat_2_1], "o-.b", markeredgecolor = 'black' )
    ax.plot( [Long_3_0, Long_3_1], [Lat_3_0, Lat_3_1], "o-.b", markeredgecolor = 'black' )

In [12]:
plt.figure()
ax = plt.subplot(111)

plotLongLatAndTrackReferences( ax )

plt.title("Navigation data and track references 2017_09_22 09h00-10h00")

### Check the existence of record files

In [13]:
firstRecord = 12
lastRecord = 7734
aux = np.zeros(7734)
fileList = []
dataDir = "/home/pleroy/DATA/PoSAR-MC_PIMA_TEST-2/2017_09_22_11_43_45"
for k in range(firstRecord, lastRecord, blocksPerFile):
    filename = dataDir + "/record" + str(k) + ".bin"
    ret = os.path.isfile(filename)
    if ret == False:
        fileList.append(0)
    else:
        fileList.append(1)
idxFileList = np.arange( len( fileList ) ) * blocksPerFile + firstRecord

In [14]:
plt.plot(idxFileList, fileList, 'o-')

In [15]:
# times recorded by Windows XP
t_0    = (11, 43, 45)
t_12   = (11, 43, 48)
t_18   = (11, 43, 49)
t_1686 = (11, 48,  6)
t_1818 = (11, 48, 26)
t_1866 = (11, 48, 32)
t_2034 = (11, 48, 51)
t_2052 = (11, 48, 54)
t_2106 = (11, 49,  2)
t_2448 = (11, 49, 55)
t_4140 = (11, 54,  7)
t_4710 = (11, 55, 33)
t_6684 = (12,  0, 29)
t_7734 = (12,  3,  6)

record0Start = 12
record0Stop = 2052

record1Start = 2106
record1Stop = 2448

record2Start = 4140
record2Stop = 4710

record3Start = 6684
record3Stop = 7734

Look for the time data corresponding to the records

In [16]:
hourOffset = 2

#timeStamp status year month day hour minute second nanoSecond gpsTimeOfWeek
idx_h = 5
idx_m = 6
idx_s = 7
idx_nano = 8
utc_h_1, utc_m_1, utc_s_1, utc_nano_1 = np.loadtxt( logUtcData_1, skiprows = 1, 
                                           usecols = (idx_h, idx_m, idx_s, idx_nano), unpack=True )
utc_h_2, utc_m_2, utc_s_2, utc_nano_2 = np.loadtxt( logUtcData_2, skiprows = 1, 
                                           usecols = (idx_h, idx_m, idx_s, idx_nano), unpack=True )
utc_h = np.concatenate( (utc_h_1, utc_h_2) )
utc_m = np.concatenate( (utc_m_1, utc_m_2) )
utc_s = np.concatenate( (utc_s_1, utc_s_2) )
utc_nano = np.concatenate( (utc_nano_1, utc_nano_2) )
utc_seconds = (utc_h + hourOffset) * 3600 + utc_m * 60 + utc_s + utc_nano * 1e-9

In [20]:
def getTimeOfDay( seconds ):
    h = np.floor( seconds / 3600 )
    m = np.floor( (seconds - h *3600) / 60 )
    s = seconds - h * 3600 - m * 60
    return (h, m, s)

def getSeconds( timeOfDay ):
    return (timeOfDay[0] * 3600 + timeOfDay[1] * 60 + timeOfDay[2])

def getIndices( start, stop ):
    idx_start = np.amax( np.where( utc_seconds <= getSeconds(start) ) )
    idx_stop = np.amax( np.where( utc_seconds <= getSeconds(stop) ) )
    return (idx_start, idx_stop)

In [18]:
# compute time of day using the date of block 0 and a computed delta

record0_tStart = getTimeOfDay( getSeconds( t_0 ) + record0Start / blocksPerFile * T_files )
record0_tStop = getTimeOfDay( getSeconds( t_0 ) + record0Stop / blocksPerFile * T_files )

record1_tStart = getTimeOfDay( getSeconds( t_0 ) + record1Start / blocksPerFile * T_files )
record1_tStop = getTimeOfDay( getSeconds( t_0 ) + record1Stop / blocksPerFile * T_files )

record2_tStart = getTimeOfDay( getSeconds( t_0 ) + record2Start / blocksPerFile * T_files )
record2_tStop = getTimeOfDay( getSeconds( t_0 ) + record2Stop / blocksPerFile * T_files )

record3_tStart = getTimeOfDay( getSeconds( t_0 ) + record3Start / blocksPerFile * T_files )
record3_tStop = getTimeOfDay( getSeconds( t_0 ) + record3Stop / blocksPerFile * T_files )

In [19]:
record0_idx = getIndices (record0_tStart, record0_tStop)
record1_idx = getIndices (record1_tStart, record1_tStop)
record2_idx = getIndices (record2_tStart, record2_tStop)
record3_idx = getIndices (record3_tStart, record3_tStop)

### Plot navigation data and record periods

In [21]:
def addOnPlot(ax, idx, color, label=''):
    idxStart = idx[0]
    idxStop = idx[1]
    ax.plot( Long[idxStart:idxStop], Lat[idxStart:idxStop], '--', color=color, label=label )
    ax.plot( Long[idxStart], Lat[idxStart], 'D', color=color, markeredgecolor = 'black' )
    ax.plot( Long[idxStop], Lat[idxStop], 'D', color=color, markeredgecolor = 'black'  )

In [22]:
def addSpot(ax, idx, color):
    ax.plot( Long[idx], Lat[idx], 'D', color=color, markeredgecolor = 'black' )

In [23]:
spot0 = getTimeOfDay( getSeconds( t_0 ) + 84 / blocksPerFile * T_files )
spot1 = getTimeOfDay( getSeconds( t_0 ) + 144 / blocksPerFile * T_files )
idx_spot0 = np.amax( np.where( utc_seconds <= getSeconds(spot0) ) )
idx_spot1 = np.amax( np.where( utc_seconds <= getSeconds(spot1) ) )
velocity_spot0_spot1 = np.average( Vel[idx_spot0:idx_spot1] )
altitude_spot0_spot1 = np.average( Alt[idx_spot0:idx_spot1] )

In [24]:
spot0

(11.0, 43.0, 57.599999999998545)

In [25]:
print(velocity_spot0_spot1)

39.0054229661


In [26]:
print(altitude_spot0_spot1)

239.896377946


In [27]:
spot1

(11.0, 44.0, 6.5999999999985448)

In [47]:
plt.figure()

ax = plt.subplot(111)

plotLongLatAndTrackReferences( ax )
plotRunaway( ax )

addOnPlot( ax, record0_idx, 'yellow', 'record 0' )
addOnPlot( ax, record1_idx, 'red', 'record 1' )
addOnPlot( ax, record2_idx, 'magenta', 'record 2' )
addOnPlot( ax, record3_idx, 'pink', 'record 3' )

addSpot( ax, idx_spot0, 'white' )
addSpot( ax, idx_spot1, 'white' )

plt.title("Navigation data, track references, record locations\nFlight 2017_09_22 09h00-10h00")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.legend()
plt.grid()

## Track 1

In [29]:
plt.figure()
ax1 = plt.subplot(211)
ax1.plot(Long, '.')
ax1.plot(np.ones(Long.shape) * runaway[1][1])
ax1.plot(np.ones(Long.shape) * runaway[0][1])
ax2 = plt.subplot(212, sharex=ax1)
ax2.plot(Lat, '.')
ax2.plot(np.ones(Lat.shape) * runaway[1][0])
ax2.plot(np.ones(Lat.shape) * runaway[0][0])

### Flight over 1

In [33]:
idx_tr1_fo1 = (320146, 322010)

In [32]:
plt.figure()
ax = plt.subplot(111)

plotLongLatAndTrackReferences( ax )

addOnPlot( ax, idx_tr1_fo1, 'red', 'record 0' )

plt.title("Navigation data and track references 2017_09_22 09h00-10h00\nTrack 1 Flight over 1")

### Flight over 2

In [34]:
idx_tr1_fo2 = (373880, 375450)

In [35]:
plt.figure()
ax = plt.subplot(111)

plotLongLatAndTrackReferences( ax )

addOnPlot( ax, idx_tr1_fo2, 'red', 'record 0' )

plt.title("Navigation data and track references 2017_09_22 09h00-10h00\nTrack 1 Flight over 2")

### Along track 1 reference

In [ ]:
plt.figure()
ax1 = plt.subplot(211)
ax1.plot(Long, '.')
ax1.plot(np.ones(Long.shape) * Long_1_0)
ax1.plot(np.ones(Long.shape) * Long_1_1)
ax2 = plt.subplot(212, sharex=ax1)
ax2.plot(Lat, '.')
ax2.plot(np.ones(Lat.shape) * Lat_1_0)
ax2.plot(np.ones(Lat.shape) * Lat_1_1)

In [36]:
idx_allTr1_fo1 = (314883, 327129)

In [37]:
plt.figure()
ax = plt.subplot(111)

plotLongLatAndTrackReferences( ax )

addOnPlot( ax, idx_allTr1_fo1, 'red', 'record 0' )

plt.title("Navigation data and track references 2017_09_22 09h00-10h00\nTrack 1 Flight over 1")

### Track 1 reference / Flight over 2

In [38]:
idx_allTr1_fo2 = (369513, 380159)

In [39]:
plt.figure()
ax = plt.subplot(111)

plotLongLatAndTrackReferences( ax )

addOnPlot( ax, idx_allTr1_fo2, 'red', 'record 0' )

plt.title("Navigation data and track references 2017_09_22 09h00-10h00\nTrack 1 Flight over 2")

## Track 3

In [50]:
plt.figure()
ax1 = plt.subplot(211)
ax1.plot(Long, '.')
ax1.plot(np.ones(Long.shape) * track3_Long_0)
ax1.plot(np.ones(Long.shape) * track3_Long_1)
ax2 = plt.subplot(212, sharex=ax1)
ax2.plot(Lat, '.')
ax2.plot(np.ones(Lat.shape) * track3_Lat_0)
ax2.plot(np.ones(Lat.shape) * track3_Lat_1)

### Track 3 reference / Flight over 1

In [51]:
idx_allTr3_fo1 = (515398, 527835)

In [62]:
# 6684 / 7734
start = 6690
stop = start + 6 * 15
spot3_0 = getTimeOfDay( getSeconds( t_0 ) + start / blocksPerFile * T_files )
spot3_1 = getTimeOfDay( getSeconds( t_0 ) + stop / blocksPerFile * T_files )
idx_spot3_0 = np.amax( np.where( utc_seconds <= getSeconds(spot3_0) ) )
idx_spot3_1 = np.amax( np.where( utc_seconds <= getSeconds(spot3_1) ) )
velocity_spot3_0_spot3_1 = np.average( Vel[idx_spot3_0:idx_spot3_1] )
altitude_spot3_0_spot3_1 = np.average( Alt[idx_spot3_0:idx_spot3_1] )

In [60]:
plt.figure()
ax = plt.subplot(111)

plotLongLatAndTrackReferences( ax )
plotRunaway( ax )

addOnPlot( ax, idx_allTr3_fo1, 'red', 'record 0' )

addSpot( ax, idx_spot3_0, 'white' )
addSpot( ax, idx_spot3_1, 'white' )

plt.title("Navigation data and track references 2017_09_22 09h00-10h00\nTrack 1 Flight over 2")

# Data selection

In [40]:
def getBlockNumberFromUtcSeconds( seconds, t_0 ):
    delta = seconds - getSeconds(t_0)
    return int( np.floor( delta / T_files ) * blocksPerFile )

In [41]:
def getAltitudeVelocity( Alt, Vel, idx0, idx1 ):
    altitude = np.average( Alt[ idx0 : idx1 ] )
    velocity = np.average( Vel[ idx0 : idx1 ] )
    print("(altitude, velocity) = ({:.3f}, {:.3f})".format(altitude, velocity) )

In [42]:
def getSelection(utc_seconds, Alt, Vel, idx):
    (idx0, idx1) = idx
    
    selection0 = getBlockNumberFromUtcSeconds( utc_seconds[idx0], t_0 )
    print( "Data selection starts at file: " + '{}'.format(selection0) )
    
    selection1 = getBlockNumberFromUtcSeconds( utc_seconds[idx1], t_0 )
    nb = int( np.floor( (selection1 - selection0) / blocksPerFile ) )
    print( "Data selection stops at file: " + '{}'.format(selection1) 
          + ", number of files in the selection: " + '{}'.format(nb+1) )
    
    getAltitudeVelocity( Alt, Vel, idx0, idx1 )

## Track 1

### Runaway Flight over 1

In [43]:
getSelection(utc_seconds, Alt, Vel, idx_tr1_fo1 )

Data selection starts at file: 24
Data selection stops at file: 84, number of files in the selection: 11
(altitude, velocity) = (239.759, 37.730)


### Runaway Flight over 2

In [44]:
getSelection(utc_seconds, Alt, Vel, idx_tr1_fo2 )

Data selection starts at file: 1818
Data selection stops at file: 1866, number of files in the selection: 9
(altitude, velocity) = (227.751, 44.205)


### Track 1 Flight over 1

In [45]:
getSelection(utc_seconds, Alt, Vel, idx_allTr1_fo1 )

Data selection starts at file: -150
Data selection stops at file: 258, number of files in the selection: 69
(altitude, velocity) = (236.732, 38.151)


### Track 1 Flight over 2

In [46]:
getSelection(utc_seconds, Alt, Vel, idx_allTr1_fo2 )

Data selection starts at file: 1668
Data selection stops at file: 2022, number of files in the selection: 60
(altitude, velocity) = (232.441, 43.748)


### Track 3 Flight over 1

In [63]:
getSelection(utc_seconds, Alt, Vel, (idx_spot3_0, idx_spot3_1) )

Data selection starts at file: 6684
Data selection stops at file: 6774, number of files in the selection: 16
(altitude, velocity) = (801.705, 36.686)


# Annexe

### Check the dates of the recorded files

In [ ]:
def getLastLine( filename ):
    with open(filename, newline='') as f:
        reader = csv.reader(f)
        for row in reader:
            last = next(reader)
        print(last)

In [ ]:
def printUtc( index ):
    print( str(utc_h[index]) 
          + " : " + str(utc_m[index]) 
          + " : " + "{:.3f}".format(utc_s[index] + utc_nano[index] * 1e-9) )

In [ ]:
def getDataAtIndex(filename, index):
    with open(filename, newline='') as f:
        reader = csv.reader(f)
        for r in range(index+1):
            next(reader)
        val = next(reader)
    return val

In [ ]:
def getBlockNumber( t, t_0 ):
    delta = getSeconds( t ) - getSeconds(t_0)
    return int( np.floor( delta / T_files ) * blocksPerFile )

def getUtcIndex( tod ):
    return np.amax( np.where( utc_seconds <= getSeconds( tod ) ) )

def getUtcData( logUtcData, timeOfDay ):
    with open( logUtcData, newline='') as f:
        reader = csv.reader(f)
        next(reader)
        utc = np.loadtxt( io.StringIO( next(reader)[0] ))
        
        while (utc[5] < timeOfDay[0] - hourOffset):
            utc = np.loadtxt( io.StringIO( next(reader)[0] ))
            
        while (utc[6] < timeOfDay[1]):
            utc = np.loadtxt( io.StringIO( next(reader)[0] ))
            
        while (utc[7] < timeOfDay[2]):
            utc = np.loadtxt( io.StringIO( next(reader)[0] ))
            
    return utc